In [ ]:
%pip install shapely seedir

In [ ]:
import json
from pprint import pformat
from copy import deepcopy
from tempfile import TemporaryDirectory

from rastervision.pytorch_backend.examples.utils import read_stac
from rastervision.pipeline.file_system import unzip

%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Patch, Polygon as MPolygon
from shapely.geometry import shape, MultiPolygon, Polygon
from shapely.ops import unary_union
import seedir as sd

In [ ]:
stac_export_uri = 'stac-export.zip'

# STAC export structure

In [ ]:
def preview_zipfile(path):
    with TemporaryDirectory() as tmp_dir:
        unzip(path, target_dir=tmp_dir)
        s = sd.seedir(path=tmp_dir, style='lines', indent=4, first='files', printout=False)
        root = s.split('\n')[0]
        s = s.replace(root, f'{path}/')
        print(s)

In [ ]:
preview_zipfile(stac_export_uri)

# STAC catalog contents

In [ ]:
def show_parsed_stac(project_infos):
    project_infos = deepcopy(project_infos)
    for info in project_infos:
        g = shape(info['aoi_geometry'])
        info['aoi_geometry']['coordinates'] = '[[...]]'
        info['image_bbox'] = info['image_bbox'].bounds
        info['label_bbox'] = info['label_bbox'].bounds
    print(pformat(project_infos))

`read_stac()` is a util function from Raster Vision that returns a dict for each project in the STAC export.

In [ ]:
project_infos = read_stac(stac_export_uri, './tmp/inspect')

Pretty print the output of `read_stac()`:

In [ ]:
show_parsed_stac(project_infos)

# Labels and AOI

As a sanity check, we can visualize the labels that we will actually be feeding into Raster Vision for training.

In [ ]:
def geojson_to_shape(path):
    with open(path, 'r') as f:
        features = json.load(f)['features']
    polygons = [shape(f['geometry']) for f in features]
    polygons = unary_union(polygons)
    return polygons

In [ ]:
def show_labels_combined(infos):
    fig = plt.figure(figsize=(16, 24))
    ax = plt.gca()
    extent = Polygon.from_bounds(*unary_union([infos[0]['label_bbox'], infos[1]['label_bbox']]).bounds)
    ax.add_patch(MPolygon(np.array(extent.exterior), fill=None, hatch='/', alpha=.5))
    for info in infos:
        aoi_polygons = MultiPolygon(shape(info['aoi_geometry']))
        label_polygons = MultiPolygon(geojson_to_shape(info['label_uri']))
        for p in aoi_polygons:
            ax.add_patch(MPolygon(np.array(p.exterior), fc='white', ec='#777'))
        for p in label_polygons:
            ax.add_patch(MPolygon(np.array(p.exterior), fc='r', ec='r'))
    plt.axis('off')
    plt.autoscale()
    legend_patches = [
        Patch(fc='w', ec='k', hatch='/', alpha=.5, label='Unlabeled region'),
        Patch(fc='w', ec='#777', alpha=1, label='Labeled region'),
        Patch(color='r', alpha=1, label='Boat'),
    ]
    plt.legend(handles=legend_patches, loc='upper center', bbox_to_anchor=(.5, .985), fontsize='xx-large', frameon=False, ncol=3)
    fig.tight_layout(pad=0)
    plt.show()

In [ ]:
show_labels_combined(project_infos)